# Here's our N-gram model: what we have so far

In [1]:
from nltk.lm.preprocessing import pad_both_ends
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, ELEProbDist
from nltk.util import pad_sequence
from nltk.lm.preprocessing import pad_both_ends
import re

In [2]:
corpus = """By this liberty they entered into a very laudable emulation to do all of them \
what they saw did please one. If any of the gallants or ladies should say, Let us drink, \
they would all drink.  If any one of them said, Let us play, they all played.  If one said, \
Let us go a-walking into the fields they went all."""

In [3]:
# get the ngrams for a corpus
def ngrams(text, n):
    n_grams = []
    for i in range(n-1, len(tokenized_corpus)): 
        n_grams.append(tuple(tokenized_corpus[i-(n-1):i+1]))
    return n_grams



In [4]:



class NgramModel():

    
    def __init__(self, corpus, n):
        self.n = n
        tokenized_corpus = self._tokenize(corpus)
        self._ngrams = self._build_ngrams(tokenized_corpus, n)
        self._cpd = self._build_distribution(self._ngrams, n)        
        
    def _tokenize(self, corpus):
        
        tokenized_corpus = []
        
        # separate punctuation from previous word
        spaced_corpus = re.sub(r'(\w)([.,?!;:])', r'\1 \2', corpus) 
        
        # split into sentences
        sentences = spaced_corpus.split('.')
        for sentence in sentences:
            words = sentence.split() # split on whitespace
            words = [word.lower() for word in words]
            words = list(pad_both_ends(words, n=self.n))
            tokenized_corpus += words
        
        return tokenized_corpus
            
    def _build_ngrams(self, tokenized_corpus, n):
        n_grams = []
        for i in range(n-1, len(tokenized_corpus)): 
            n_grams.append(tuple(tokenized_corpus[i-(n-1):i+1]))    
        return n_grams
    
    def _build_distribution(self, corpus, n):
               
        cfd = ConditionalFreqDist()
        for ngram in self._ngrams:
            condition = tuple(ngram[0:n-1]) 
            outcome = ngram[n-1]
            
            cfd[condition][outcome] += 1
        bins = len(cfd) # we have to pass the number of bins in our freq dist in as a parameter to probability distribution, so we have a bin for every word
        cpd = ConditionalProbDist(cfd, ELEProbDist, bins)
        self.cpd = cpd
        return cpd
        
    def generate(self, num_sentences = 1, seed = []):
        """
        There are two cases to deal with here. Either we have a start string, or we don't. 
        If we are given a start string, we'll have to find the last n-1 gram and condition on that
        If we are not, we need to generate the first n-1 gram. For a trigram model, we need a bigram. But how can we use our model to generate new words when we have fewer than two words to condition on?
        We can use a bigram model! But wait. If we have a bigram model, how do we generate the first token without another token to condition on? 
        We can use a unigram model! 
        Recursion will save us here. Turns out the easiest way to do this will be to recursively construct an n-1gram model and store it in the main model.
        And how can we 
        Either way, we need a seed condition to enter into the loop with.
        """

        # place to put generated tokens
        string = []

        if seed:
            string = string + (list(pad_sequence(seed, self.n, pad_left=True, pad_right=False, left_pad_symbol='<s>') ) )
        else:
            string = string + (list(pad_sequence('', self.n, pad_left=True, pad_right=False, left_pad_symbol='<s>') ) )
        
        for i in range(num_sentences):
            next_token = tuple(string[-(self.n-1):])
            
            # keep generating tokens as long as we havent reached the stop sequence
            while next_token != '</s>':
                
                # get the last n-1 tokens to condition on next
                lessgram = tuple(string[-(self.n-1):])

    
                next_token = self.cpd[lessgram].generate()
                string.append( next_token )

        string = ' '.join(string)

        return string

        
        

# Scaling up

If we keep increasing n, our generated text starts to repeat our input text almost word for word. To get interesting behavior, we have to increase the size of the corpus. Let's try with a much bigger corpus!

In [5]:
import nltk
!python3 -m nltk.downloader gutenberg


<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/gabriellachronis/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


NLTK comes with several built in corpora, including a selection of books from project gutenberg

In [6]:
# import corpus using an alias to avoid namespace confustion with our corpus variable
from nltk import corpus as corpiss 

corpiss.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [7]:
kjv = corpiss.gutenberg.words('bible-kjv.txt')
list(kjv)[:40]

['[',
 'The',
 'King',
 'James',
 'Bible',
 ']',
 'The',
 'Old',
 'Testament',
 'of',
 'the',
 'King',
 'James',
 'Bible',
 'The',
 'First',
 'Book',
 'of',
 'Moses',
 ':',
 'Called',
 'Genesis',
 '1',
 ':',
 '1',
 'In',
 'the',
 'beginning',
 'God',
 'created',
 'the',
 'heaven',
 'and',
 'the',
 'earth',
 '.',
 '1',
 ':',
 '2',
 'And']

### A Few final adjustments

We have some housekeeping things to take care of. 

1. Because we have encoded sentence breaks as a string of start and stop sequences, we now will generate a lot of them in our output. We add function to strip them out, and update our generate method to strip out these tokens before printing



In [8]:
def add_stops(string):
    """
    function to convert the stop/start sequence back into periods.
    strips all the sequences of any number of stop tokens followed by the some number of start tokens
    and replaces them with a period.

    then strips any remaining stop and start sequences (which will occur at the beginning and end of our entire generated sequence)
    """
    string = re.sub(r"</s>(?:\s</s>)*\s<s>(?:\s<s>)*", ".", string)

    string = re.sub(r"(<s>\s)+", "", string) # initial tokens
    string = re.sub(r"(</s>)", "", string) # final token

    return string


test_string = '<s> <s> <s> take mark , and see now , and humble ye them , and seethe his flesh in running water , and be slain , and they rode upon the camels , and have washed their robes , and made unto themselves of the holy angels , and to solomon his son </s> </s> </s> <s> <s> <s> even so , father ; for the press is full , the fats overflow ; for their wickedness </s> </s> </s> <s> <s> <s> 119 : 59 i thought on my ways , as a seal upon him , and would none of my words </s> </s> </s> <s> <s> <s> 30 : 37 and jacob took him rods of green poplar , and of beast : it is most holy unto him of his labour the days of jehoahaz </s> </s> </s> <s> <s> <s> 97 : 3 a man shall dig a pit , and sold joseph to the ishmeelites for twenty pieces of silver out of the sheath thereof , and add unto it the fifth part unto pharaoh , and say to hezekiah , thus saith god the lord , after this manner therefore pray ye : our father which art in heaven , now is the judgment of moab </s> </s> </s> <s> <s> <s> 134 : 3 the aged women likewise , that they escaped all safe to land </s> </s> </s> <s> <s> <s> spots they are and blemishes , sporting themselves with their own works , as god liveth , who hath begotten me these , seeing i have rejected him from reigning over israel ? 15 : 37 and reuben spake unto his brother a name in israel , telleth the king of lachish , bind the tire of thine head upon thee , until it was a river that i could withstand god ? 11 : 30 are they not written in this book : worship god : for man would swallow me up </s> </s> </s> <s> <s> <s> unto him that giveth his neighbour drink , that puttest thy bottle to him , rise , and measure the temple of babylon , my servant deceived me : my moisture is turned into mourning </s> </s> </s> <s> <s> <s> 106 : 18 and samuel told him every whit , and hid snares for my feet </s> </s> </s> <s> <s> <s>'
model = NgramModel(corpus, 3)
add_stops(test_string)

'take mark , and see now , and humble ye them , and seethe his flesh in running water , and be slain , and they rode upon the camels , and have washed their robes , and made unto themselves of the holy angels , and to solomon his son . even so , father ; for the press is full , the fats overflow ; for their wickedness . 119 : 59 i thought on my ways , as a seal upon him , and would none of my words . 30 : 37 and jacob took him rods of green poplar , and of beast : it is most holy unto him of his labour the days of jehoahaz . 97 : 3 a man shall dig a pit , and sold joseph to the ishmeelites for twenty pieces of silver out of the sheath thereof , and add unto it the fifth part unto pharaoh , and say to hezekiah , thus saith god the lord , after this manner therefore pray ye : our father which art in heaven , now is the judgment of moab . 134 : 3 the aged women likewise , that they escaped all safe to land . spots they are and blemishes , sporting themselves with their own works , as god 

2. numbers are a problem for n-gram models becayse there are so many of them. we don't want to eliminate them, because they are meaningful, but we want to abstract away from the individual numbers. In addition, we might want to get rid of some other things like parentheticals and quotes, becayse these impossible for our model to keep track of given it's amount of memory. We can take care of these things in the preprocessing function

In [9]:
from functools import reduce

def _tokenize(self, corpus):
    # The list of regular expressions and replacements to be applied
    # the order here matters! these replacements will happen in order
    replacements = [
         ["[-\n]",                   " "] # Hyphens to whitespace
        ,[r'[][(){}#$%"]',           ""] # Strip unwanted characters like quotes and brackets
        ,[r'\s([./-]?\d+)+[./-]?\s', " [NUMBER] "] # Standardize numbers
        ,[r'\.{3,}',                 " [ELLIPSIS] "] # remove ellipsis
        ,[r'(\w)([.,?!;:])',         r'\1 \2' ]  # separate punctuation from previous word
    ]

    # This is a function that applies a single replacement from the list
    resub = lambda words, repls: re.sub(repls[0], repls[1], words)

    # we use the resub function to applea each replacement to the entire corpus,
    normalized_corpus = reduce(resub, replacements, corpus)


    sentences = normalized_corpus.split('.')

    tokens = []
    for sentence in sentences:
        words = sentence.split() # split on whitespace
        words = [word.lower() for word in words]
        words = list(pad_both_ends(words, n=self.n))
        tokens += words

    return tokens

Here is a final version of our class with all the bells and whistles

In [10]:
from nltk.lm.preprocessing import pad_both_ends
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, ELEProbDist
from nltk.util import pad_sequence
from nltk.lm.preprocessing import pad_both_ends
from functools import reduce

class NgramModel():

    
    def __init__(self, corpus, n):
        self.n = n
        tokenized_corpus = self._tokenize(corpus)
        self._ngrams = self._build_ngrams(tokenized_corpus, n)
        self._cpd = self._build_distribution(self._ngrams, n)        

    def _tokenize(self, corpus):
        # The list of regular expressions and replacements to be applied
        # the order here matters! these replacements will happen in order
        replacements = [
             ["[-\n]",                   " "] # Hyphens to whitespace
            ,[r'[][(){}#$%"]',           ""] # Strip unwanted characters like quotes and brackets
            ,[r'\s([./-]?\d+)+[./-]?\s', " [NUMBER] "] # Standardize numbers
            ,[r'\.{3,}',                 " [ELLIPSIS] "] # remove ellipsis
            ,[r'(\w)([.,?!;:])',         r'\1 \2' ]  # separate punctuation from previous word
        ]
        
        # This is a function that applies a single replacement from the list
        resub = lambda words, repls: re.sub(repls[0], repls[1], words)
        
        # we use the resub function to applea each replacement to the entire corpus,
        normalized_corpus = reduce(resub, replacements, corpus)
        
        
        sentences = normalized_corpus.split('.')
        
        tokens = []
        for sentence in sentences:
            words = sentence.split() # split on whitespace
            words = [word.lower() for word in words]
            words = list(pad_both_ends(words, n=self.n))
            tokens += words
        
        return tokens
            
    def _build_ngrams(self, tokenized_corpus, n):
        n_grams = []
        for i in range(n-1, len(tokenized_corpus)): 
            n_grams.append(tuple(tokenized_corpus[i-(n-1):i+1]))    
        return n_grams
    
    def _build_distribution(self, corpus, n):
               
        cfd = ConditionalFreqDist()
        for ngram in self._ngrams:
            condition = tuple(ngram[0:n-1]) 
            outcome = ngram[n-1]
            
            cfd[condition][outcome] += 1
        bins = len(cfd) # we have to pass the number of bins in our freq dist in as a parameter to probability distribution, so we have a bin for every word
        cpd = ConditionalProbDist(cfd, ELEProbDist, bins)
        self.cpd = cpd
        return cpd
        
    def generate(self, num_sentences = 1, seed = []):
        """
        There are two cases to deal with here. Either we have a start string, or we don't. 
        If we are given a start string, we'll have to find the last n-1 gram and condition on that
        If we are not, we need to generate the first n-1 gram. For a trigram model, we need a bigram. But how can we use our model to generate new words when we have fewer than two words to condition on?
        We can use a bigram model! But wait. If we have a bigram model, how do we generate the first token without another token to condition on? 
        We can use a unigram model! 
        Recursion will save us here. Turns out the easiest way to do this will be to recursively construct an n-1gram model and store it in the main model.
        And how can we 
        Either way, we need a seed condition to enter into the loop with.
        """

        # place to put generated tokens
        string = []

        if seed:
            string = string + (list(pad_sequence(seed, self.n, pad_left=True, pad_right=False, left_pad_symbol='<s>') ) )
        else:
            string = string + (list(pad_sequence('', self.n, pad_left=True, pad_right=False, left_pad_symbol='<s>') ) )
        
        for i in range(num_sentences):
            next_token = tuple(string[-(self.n-1):])
            
            # keep generating tokens as long as we havent reached the stop sequence
            while next_token != '</s>':
                
                # get the last n-1 tokens to condition on next
                lessgram = tuple(string[-(self.n-1):])

    
                next_token = self.cpd[lessgram].generate()
                string.append( next_token )

        string = ' '.join(string)
        string = add_stops(string)

        return string

    
    def add_stops(string):
        """
        function to convert the stop/start sequence back into periods.
        strips all the sequences of any number of stop tokens followed by the some number of start tokens
        and replaces them with a period.

        then strips any remaining stop and start sequences (which will occur at the beginning and end of our entire generated sequence)
        """
        string = re.sub(r"</s>(?:\s</s>)*\s<s>(?:\s<s>)*", ".", string)

        string = re.sub(r"(<s>\s)+", "", string) # initial tokens
        string = re.sub(r"(</s>)", "", string) # final token

        return string

In [11]:
model = NgramModel(corpus, 2)


In [12]:
model.generate(10)

'. if any one of the gallants or ladies should say , let us play , let us play , let us drink , they all drink , let us play , let us go a walking into a very laudable emulation to do all . by this liberty they would all of the gallants or ladies should say , let us go a walking into a walking into a walking into the fields they entered into the gallants or ladies should say , let us play , they would all . by this liberty they saw did please one of them what they went all . if any of them said , they all . . if any of the gallants or ladies should say , let us go a walking into the gallants or ladies should say , they would all of them what they all of them what they entered into a very laudable emulation to do all . if any of the fields they saw did please one of them what they all of the fields they saw did please one of them said , let us play , they would all . if one said , let us drink . '

We try generating a 4-gram model with the King James Bible

Our model expects its training corpus in the form of a single string.

In [13]:
kjv = (' ').join(kjv)

In [14]:
model = NgramModel(kjv, 4)


In [15]:
model.generate(10)

'ezra [number] : [number] zadok his son , take now thy son , he shall know that my redeemer liveth , and thy patience , and thy fairs , thy merchandise , and all knowledge ; [number] : [number] declare his glory among the gentiles to forsake moses , saying that he himself had dedicated , into the hand of esau : for i speak not this to condemn you : for he said to gehazi , gird up thy loins , and mourned for his son . his glory covered the heavens , jesus the son of melea , which was , and samuel was laid down to sleep ; [number] : [number] scornful men bring a city into a snare : but wise men turn away wrath . fare ye well . destroy not him with thy robe , and killed him '

# Let's do a mashup

Intro to beautiful soup for scraping web text

In [16]:
!pip3 install beautifulsoup4

from bs4 import *

import requests

url = 'https://theanarchistlibrary.org/library/the-invisible-committe-now.muse'
res = requests.get(url)
html_page = res.text

# Parse the source code using BeautifulSoup
soup = BeautifulSoup(html_page, 'html.parser')

# Extract the plain text content
text = soup.get_text()

# Print the plain text
print(text[:2000])



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
#cover t-i-the-invisible-committe-now-8.png
#pubdate 2018-02-25 19:37:42 +0000
#title Now
#author The Invisible Committe
#LISTtitle Now
#SORTauthors comité invisible
#date 2017
#source Retrived on February 18, 2018 from https://illwilleditions.noblogs.org/files/2018/02/Invisible-Committee-NOW-READ.pdf
#lang en
#SORTtopics insurrectionary, communization
#notes The Invisible Committee are an anonymous fragment of the Imaginary Party.
First published as *Maintenant* in May, 2017.
Translated by Robert Hurley.


No more waiting.
No more hoping.
No more letting ourselves be distracted, unnerved.
Break and enter.
Put untruth back in its place.
Believe in what we feel.
Act accordingly.
Force our way into the present.
Try. Fail this time. Try again. Fail better.
Persist. Attack. Build.
Go down one’s road.
Win perhaps.
In any case, overcome.
Live, therefore.
Now...


** 

In [17]:
print(len(kjv))
print(len(text))
print(len(text * 20))

4495207
199704
3994080


In [18]:

mashup = text * 20 + kjv 


In [19]:
model = NgramModel(mashup, 2)


In [20]:
model.generate(5)

'those in the happiness , and under de mayerne , [number] : [number] : rottenness in which nullifies all , so that endureth continually ? watchman went into thine handmaidens came in jonathan ? why sleep the sovereignty of course was malchus . so can it on in beersheba , he be fully persuaded of a vast police officers stood among yourselves off before the hall again seven sabbaths , and iron pillar after men they may breed abundantly i chose israel is proud wrath ; and the decree which he return . [number] and it caused an unorganized chaos of life , and get him lie one or by the video of us swallow it were still are plainly of triumph . ” pierre peuchmaurd , they kept by the 1960s , and david stooped for the land of his fury of sabotage of omri king , so abijah pursued mine adversity consider ; and sheba gave him in natural branches of life of lies against any point to keep house also came the laws thereof ninety and flourishing in egypt riseth against yourselves ? of art angry man co

Wow!

'among the hills that are weaned from the waters saw thee polluted in thy glory above all people , from beersheba to mount up with . [number] : [number] and shaalabbin , and partly broken . report , that jehoshaphat the king is among us still believed in hope ; patient in spirit ; and half of thy power preserve thou those that served in the womb : if jacob take a lump of figs were set there upon him shall inherit all things thereon . in most militants this search for my gold and the lord separated the sons shall eat clean provender , which loveth thee and abishai , and kings have had dominion over our cattle . then he sacrificed also and to whomsoever he will prosper us ; thus have been occupied therein . yellowed figures of cherubims and palm trees : they serve not thy left side , upon their altars : but according to our hand be upon every fowl of the european union . all these did moses command joshua , this do ye look on us ; because a deep sleep fell upon it before saul : [number] wise men , let them turn their mourning . after theo’s rape , a strong wind ? [number] : [number] open thou mine affliction . to him remaining . rather comically , he took counsel how they might attain to innocency ? [number] : [number] for behold the place hormah  '

# Exercise / Homework??

Make a mashup of two texts. They can be texts you wrote (a collection of tweets, an essay), or from anywhere. You can use libgen to find books and Calibre to convert them to text. Either paste the text directly into a notebook or use a Python utility for reading files.

In [21]:
alice = (' ').join(corpiss.gutenberg.words('carroll-alice.txt'))
now = text

print(len(alice))
print(len(text))

150118
199704


In [22]:
mashups = alice + text
model = NgramModel(mashups, 2)


In [23]:
model.generate(5)

"his eye fell on march 2015 , still can’t picture . economy until they’ve just , schizophrenia . it’s well eh , mostly kings and falls apart from all crowded together : its prevailing mood . in reply , with zadists and still managed ? alice coming into the flurry of vagabonds , you please come and down its wine ,' continued the table and humans to twist it scatters purposes . ' what porpoise , down a theory of these days "

# References

most used: 
* https://notebook.community/luketurner/ipython-notebooks/notebooks/n-gram%20tutorial
* https://medium.com/analytics-vidhya/a-comprehensive-guide-to-build-your-own-language-model-in-python-5141b3917d6d
* https://towardsdatascience.com/simulating-text-with-markov-chains-in-python-1a27e6d13fc6

others:
* https://eliteai-coep.medium.com/building-n-gram-language-model-from-scratch-9a5ec206b520
* https://github.com/joshualoehr/ngram-language-model/blob/master/language_model.py
* http://www.pygaze.org/2016/03/how-to-code-twitter-bot/
    - code: https://github.com/esdalmaijer/markovbot
* https://towardsdatascience.com/implementing-a-character-level-trigram-language-model-from-scratch-in-python-27ca0e1c3c3f